<h1 style='background-color: BLACK; padding: 10px; color: white; text-align: center;'>ZOOPLA </h1>

<h1 style='background-color: BLACK; padding: 10px; color: white'> Sales </h1>


First things first, we will import the required libraries

In [1]:
import undetected_chromedriver as webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

Next, we will load in the dataset containing the list of postcode for this project

In [2]:
codes = pd.read_csv('London postcode districts.xlsx - PC DIST.csv')

Now, we will create a function called 'zoopla_sales' to scrap properties that are for sale on the Zoopla website

In [3]:
def zoopla_sales(postcodes, Trans_type, website, df):
    """
    This function scrapes property data from Rightmove for a given list of postcodes and transaction type,
    and returns the data as a pandas DataFrame.

    Arguments:
    postcodes -- a list of postcodes for which to scrape property data
    Trans_type -- the transaction type of the properties to be scraped ('sales' or 'rent')
    website -- the name of the website being scraped (in this case, 'Rightmove')
    df -- an empty pandas DataFrame to store the scraped data

    Returns:
    df1 -- a pandas DataFrame containing the scraped property data
    
    """    
    # Set the path to the ChromeDriver executable
    s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")
    
    # Launch the ChromeDriver with the specified service
    driver = webdriver.Chrome(service= s)
    
    # Navigate to the specified URL 
    driver.get('https://www.zoopla.co.uk/') 

    # Maximize the window
    time.sleep(2)
    driver.maximize_window()

    # Find the search bar  
    time.sleep(2.2)
    #search = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[1]/div/div/div/div/div/div/input')
    try:
        search = driver.find_element(By.XPATH, "//input[contains(@id,'downshift')]")
        search.send_keys(postcode)
        time.sleep(1.3)
    except:
        search = driver.find_element(By.XPATH, "//input[contains(@class,'_1qzmny55 _1ftx2fq8')]")
        search.send_keys(postcode)
        time.sleep(1.3)
    
    
    # Click the search button
    time.sleep(1.5)   
    driver.find_element(By.XPATH, "//button[@class='x8jo560 x8jo562 x8jo56a _1ftx2fq8'][1]").click()
    
    
    
    # Initialize empty lists for storing scraped data
    Trans_type = []
    address = []
    types = []
    bedrooms = []
    bathrooms = []
    prices = []
    desc = []
    date_added = []
    agent_list = []
    property_url = []
    website = []
    
    # Create an empty dataframe to store the scraped data
    df = pd.DataFrame()
    
     # Setting the page number to be 1 
    i = 1
    while True: 
        # Scrapping data for the required features in the first page
        time.sleep(1) 
        print("{} {} {} {}".format('scraping page', i,'from', postcode ))
        
        time.sleep(2.1)
        try: 
            driver.find_element(By.XPATH, "//*[contains(@class,'u94mg1 u94mg4 u94mg9')]").click()
            time.sleep(1.5)
            
        except:
            time.sleep(1.0)
            
        time.sleep(1.4)    
        address_list =driver.find_elements(By.XPATH, "//h2[contains(@class,'_1ankud51 _1ftx2fq8')]")
        type_list =driver.find_elements(By.XPATH, "//h3[contains(@class,'_1ankud52 _1ftx2fq9')]")
        bedroom_list =driver.find_elements(By.XPATH, "//div[contains(@id,'listing')]//li[1]//span[2]")
        bathroom_list =driver.find_elements(By.XPATH, "//div[contains(@id,'listing')]//li[3]//span[2]")
        price_list =driver.find_elements(By.XPATH, "//p[contains(@class,'_170k6632 _1ftx2fq6')]")
        desc_list =driver.find_elements(By.XPATH, "//p[contains(@class,'_1ankud53 _1ftx2fq9')]")
        date_added_list =driver.find_elements(By.XPATH, "//li[contains(@class,'_18cib8e1')]")
        agent_list_list =driver.find_elements(By.XPATH, "//img[contains(@class,'_12bxhf70')]")
        property_url_list = driver.current_url
        Trans_type_list = Trans_type
        website_list = website
        for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
            address.append(address_item.text)
            types.append(type_item.text)
            bedrooms.append(bedroom_item.text)
            bathrooms.append(bathroom_item.text)
            prices.append(price_item.text)
            desc.append(desc_item.text)
            date_added.append(date_added_item.text)
            agent_list.append(agent_list_item.text)
            property_url.append(property_url_list)
            Trans_type.append(Trans_type_list)
            website.append(website_list)


        time.sleep(1.3)
        # get the height of the page
        page_height = driver.execute_script("return document.body.scrollHeight;")

        
        # Cancel the pop-up window
        j= i + 1
        url = f'https://www.zoopla.co.uk/for-sale/property/{postcode}/?q={postcode}&search_source=home&pn={j}'
        if len(address_list) or len(type_list) != 0:
            driver.get(url)
        else:
            print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)
            break
        
            
        #Increment the page number
        i += 1
        time.sleep(1)
                
        
    # Create a dataframe to store data scrapped for each postcode    
    df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Sales','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'Zoopla', 'listing_URL':property_url})
    df1.insert(0, 'Unique_Id', [f'{postcode}S{i+1:05d}ZP' for i in range(len(df1))])
    print('Total numbers of properties available in ' + postcode + ' is ' + str(df1.shape[0]))

     # Concat the dataframe obtain for all postcodes
    df = pd.concat([df, df1], ignore_index=True)
    
     # Return a dataframe
    return df1



Now, the data for properties available for sale at Zoopla is scrapped

In [ ]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = zoopla_sales(postcode, 'Sales', 'Zoopla', df)
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('------------------------ WEB SCRAPING COMPLETED: OVERALL TOTAL NUMBER OF PROPERTIES SCRAPED IS ' + str(df.shape[0]))

In case the web scraping breaks while running due to network, we can continue the scrapping process from the last postcode as follows

In [19]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes.loc[256:]['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = zoopla_sales(postcode, 'Sales', 'Zoopla', df)
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('------------------------ WEB SCRAPING COMPLETED: OVERALL TOTAL NUMBER OF PROPERTIES SCRAPED IS ' + str(df.shape[0]))



scraping page 1 from WC1B
scraping page 2 from WC1B
------------------------------- SCRAPING COMPLETED FOR WC1B
Total numbers of properties available in WC1B is 13
scraping page 1 from WC1E
scraping page 2 from WC1E
------------------------------- SCRAPING COMPLETED FOR WC1E
Total numbers of properties available in WC1E is 21
scraping page 1 from WC1H
scraping page 2 from WC1H
scraping page 3 from WC1H
scraping page 4 from WC1H
------------------------------- SCRAPING COMPLETED FOR WC1H
Total numbers of properties available in WC1H is 40
scraping page 1 from WC1N
scraping page 2 from WC1N
scraping page 3 from WC1N
------------------------------- SCRAPING COMPLETED FOR WC1N
Total numbers of properties available in WC1N is 28
scraping page 1 from WC1V
scraping page 2 from WC1V
------------------------------- SCRAPING COMPLETED FOR WC1V
Total numbers of properties available in WC1V is 10
scraping page 1 from WC1X
scraping page 2 from WC1X
scraping page 3 from WC1X
scraping page 4 from WC1

Saving the dataframe into a csv file

In [20]:
df.to_csv('Zoopla_Sales_data.csv')

Loading the dataset and removing an unwanted column

In [3]:
df_ZP_sales = pd.read_csv('Zoopla_Sales_data.csv')
df_ZP_sales = df_ZP_sales.drop('Unnamed: 0', axis=1)

In [4]:
# Check the shape of the dataset
df_ZP_sales.shape

(49521, 13)

Almost 50,000 properties were scrapped

In [5]:
df_ZP_sales

,Unique_Id,Location,Tansaction_Type,Property_Type,Address,Bedrooms,Bathrooms,Price,Description,Listing_Date,Agent,Listing_Source,listing_URL
0,BR1S00001ZP,BR1,Sales,"Bishops Avenue, Bromley, Kent BR1",6 bed semi-detached house for sale,6.0,2.0,"£1,250,000",Chain free and newly rennovated to an exceptio...,Listed on 18th May 2023,NaN,Zoopla,https://www.zoopla.co.uk/for-sale/property/br1...
1,BR1S00002ZP,BR1,Sales,"College Road, Bromley BR1",1 bed flat for sale,1.0,1.0,"£255,000",Escape the city hustle with this stunning one-...,Listed on 18th May 2023,NaN,Zoopla,https://www.zoopla.co.uk/for-sale/property/br1...
2,BR1S00003ZP,BR1,Sales,"Marian Gardens, Bromley, Kent BR1",2 bed flat for sale,2.0,1.0,"£550,000",Located in the prestigious Hampton Grange deve...,Listed on 18th May 2023,NaN,Zoopla,https://www.zoopla.co.uk/for-sale/property/br1...
3,BR1S00004ZP,BR1,Sales,"Forde Avenue, Bromley, Kent BR1",3 bed terraced house for sale,3.0,1.0,"£600,000",A very generously proportioned mid terraced th...,Listed on 18th May 2023,NaN,Zoopla,https://www.zoopla.co.uk/for-sale/property/br1...
4,BR1S00005ZP,BR1,Sales,"Ellen Close, Bickley, Bromley BR1",3 bed semi-detached house for sale,3.0,2.0,"£525,000","*Guide Price £525,000 - £550,000* A spacious t...",Listed on 18th May 2023,NaN,Zoopla,https://www.zoopla.co.uk/for-sale/property/br1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49516,WC2RS00049ZP,WC2R,Sales,"Strand, Covent Garden, London WC2R",1 bed flat for sale,1.0,1.0,"£1,500,000",A one bedroom flat for sale in the luxurious 1...,Listed on 30th Nov 2020,NaN,Zoopla,https://www.zoopla.co.uk/for-sale/property/WC2...
49517,WC2RS00050ZP,WC2R,Sales,"Strand, Covent Garden WC2R",1 bed flat for sale,1.0,1.0,"£1,500,000","A stunning one bedroom, one bathroom apartment...",Listed on 19th Nov 2020,NaN,Zoopla,https://www.zoopla.co.uk/for-sale/property/WC2...
49518,WC2RS00051ZP,WC2R,Sales,"Wren House, 190 The Strand, London WC2R",Studio for sale,1.0,1.0,"£900,000","This incredibly finished apartment, set in a q...",Listed on 17th Nov 2020,NaN,Zoopla,https://www.zoopla.co.uk/for-sale/property/WC2...
49519,WC2RS00052ZP,WC2R,Sales,"190 Strand, Strand WC2R",4 bed flat for sale,4.0,1.0,"£21,300,000",Beautiful Large Penthouse. A New Build Develop...,Listed on 11th Feb 2020,NaN,Zoopla,https://www.zoopla.co.uk/for-sale/property/WC2...


<h1 style='background-color: BLACK; padding: 10px; color: white'> Rent </h1>

Now, we will create a function called 'zoopla_rent' to scrap properties that are for rent on the Zoopla website

In [4]:
def zoopla_rent(postcodes, Trans_type, website, df):
    """
    This function scrapes property data from Rightmove for a given list of postcodes and transaction type,
    and returns the data as a pandas DataFrame.

    Arguments:
    postcodes -- a list of postcodes for which to scrape property data
    Trans_type -- the transaction type of the properties to be scraped ('sales' or 'rent')
    website -- the name of the website being scraped (in this case, 'Rightmove')
    df -- an empty pandas DataFrame to store the scraped data

    Returns:
    df1 -- a pandas DataFrame containing the scraped property data
    
    """    
    # Set the path to the ChromeDriver executable
    s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")
    
    # Launch the ChromeDriver with the specified service
    driver = webdriver.Chrome(service= s)
    
    # Navigate to the specified URL 
    driver.get('https://www.zoopla.co.uk/') 

    # Maximize the window
    time.sleep(2)
    driver.maximize_window()
    
    #Click the rent button
    time.sleep(1.5)   
    driver.find_element(By.XPATH, "//button[contains(@id,'radix-:Reckt6:-trigger-to-rent')]").click()

    # Find the search bar  
    time.sleep(2.2)
    #search = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[1]/div/div/div/div/div/div/input')
    try:
        search = driver.find_element(By.XPATH, "//input[contains(@id,'downshift')]")
        search.send_keys(postcode)
        time.sleep(1.3)
    except:
        search = driver.find_element(By.XPATH, "//input[contains(@class,'_1qzmny55 _1ftx2fq8')]")
        search.send_keys(postcode)
        time.sleep(1.3)
    
    
    # Click the search button
    time.sleep(1.5)   
    driver.find_element(By.XPATH, "//button[@class='x8jo560 x8jo562 x8jo56a _1ftx2fq8'][1]").click()
    
    
    
    # Initialize empty lists for storing scraped data
    Trans_type = []
    address = []
    types = []
    bedrooms = []
    bathrooms = []
    prices = []
    desc = []
    date_added = []
    agent_list = []
    property_url = []
    website = []
    
    # Create an empty dataframe to store the scraped data
    df = pd.DataFrame()
    
     # Setting the page number to be 1 
    i = 1
    while True: 
        # Scrapping data for the required features in the first page
        time.sleep(1) 
        print("{} {} {} {}".format('scraping page', i,'from', postcode ))
        
        time.sleep(2.1)
        try: 
            driver.find_element(By.XPATH, "//*[contains(@class,'u94mg1 u94mg4 u94mg9')]").click()
            time.sleep(1.5)
            
        except:
            time.sleep(1.0)
            
        time.sleep(1.4)    
        address_list =driver.find_elements(By.XPATH, "//h2[contains(@class,'_1ankud51 _1ftx2fq8')]")
        type_list =driver.find_elements(By.XPATH, "//h3[contains(@class,'_1ankud52 _1ftx2fq9')]")
        bedroom_list =driver.find_elements(By.XPATH, "//div[contains(@id,'listing')]//li[1]//span[2]")
        bathroom_list =driver.find_elements(By.XPATH, "//div[contains(@id,'listing')]//li[3]//span[2]")
        price_list =driver.find_elements(By.XPATH, "//p[contains(@class,'_170k6632 _1ftx2fq6')]")
        desc_list =driver.find_elements(By.XPATH, "//p[contains(@class,'_1ankud53 _1ftx2fq9')]")
        date_added_list =driver.find_elements(By.XPATH, "//li[contains(@class,'_18cib8e1')]")
        agent_list_list =driver.find_elements(By.XPATH, "//img[contains(@class,'_12bxhf70')]")
        property_url_list = driver.current_url
        Trans_type_list = Trans_type
        website_list = website
        for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
            address.append(address_item.text)
            types.append(type_item.text)
            bedrooms.append(bedroom_item.text)
            bathrooms.append(bathroom_item.text)
            prices.append(price_item.text)
            desc.append(desc_item.text)
            date_added.append(date_added_item.text)
            agent_list.append(agent_list_item.text)
            property_url.append(property_url_list)
            Trans_type.append(Trans_type_list)
            website.append(website_list)


        time.sleep(1.3)
        # get the height of the page
        page_height = driver.execute_script("return document.body.scrollHeight;")

        
        # Cancel the pop-up window
        j= i + 1        
        url = f'https://www.zoopla.co.uk/to-rent/property/{postcode}/?price_frequency=per_month&q={postcode}&search_source=home&pn={j}'
        if len(address_list) or len(type_list) != 0:
            driver.get(url)
        else:
            print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)
            break
        
            
        #Increment the page number
        i += 1
        time.sleep(1)
                
        
    # Create a dataframe to store data scrapped for each postcode    
    df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Rent','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'Zoopla', 'listing_URL':property_url})
    df1.insert(0, 'Unique_Id', [f'{postcode}R{i+1:05d}ZP' for i in range(len(df1))])
    print('Total numbers of properties available in ' + postcode + ' is ' + str(df1.shape[0]))

     # Concat the dataframe obtain for all postcodes
    df = pd.concat([df, df1], ignore_index=True)
    
     # Return a dataframe
    return df1



Now, the data for properties available for rent at Zoopla is scrapped

In [ ]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = zoopla_rent(postcode, 'Rent', 'Zoopla', df)
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('------------------------ WEB SCRAPING COMPLETED: OVERALL TOTAL NUMBER OF PROPERTIES SCRAPED IS ' + str(df.shape[0]))

In case the web scraping breaks while running due to network, we can continue the scrapping process from the last postcode as follows

In [10]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes.loc[231:]['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = zoopla_rent(postcode, 'Rent', 'Zoopla', df)
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('------------------------ WEB SCRAPING COMPLETED: OVERALL TOTAL NUMBER OF PROPERTIES SCRAPED IS ' + str(df.shape[0]))



scraping page 1 from TW6
scraping page 2 from TW6
------------------------------- SCRAPING COMPLETED FOR TW6
Total numbers of properties available in TW6 is 12
scraping page 1 from TW7
scraping page 2 from TW7
scraping page 3 from TW7
scraping page 4 from TW7
------------------------------- SCRAPING COMPLETED FOR TW7
Total numbers of properties available in TW7 is 54
scraping page 1 from TW8
scraping page 2 from TW8
scraping page 3 from TW8
scraping page 4 from TW8
scraping page 5 from TW8
scraping page 6 from TW8
scraping page 7 from TW8
scraping page 8 from TW8
scraping page 9 from TW8
scraping page 10 from TW8
------------------------------- SCRAPING COMPLETED FOR TW8
Total numbers of properties available in TW8 is 136
scraping page 1 from TW9
scraping page 2 from TW9
scraping page 3 from TW9
scraping page 4 from TW9
scraping page 5 from TW9
------------------------------- SCRAPING COMPLETED FOR TW9
Total numbers of properties available in TW9 is 63
scraping page 1 from UB1
scraping

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00DB8893+48451]
	(No symbol) [0x00D4B8A1]
	(No symbol) [0x00C55058]
	(No symbol) [0x00C3D073]
	(No symbol) [0x00C9DEBB]
	(No symbol) [0x00CABFD3]
	(No symbol) [0x00C9A0B6]
	(No symbol) [0x00C77E08]
	(No symbol) [0x00C78F2D]
	GetHandleVerifier [0x01018E3A+2540266]
	GetHandleVerifier [0x01058959+2801161]
	GetHandleVerifier [0x0105295C+2776588]
	GetHandleVerifier [0x00E42280+612144]
	(No symbol) [0x00D54F6C]
	(No symbol) [0x00D511D8]
	(No symbol) [0x00D512BB]
	(No symbol) [0x00D44857]
	BaseThreadInitThunk [0x771A00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77587B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77587B1E+238]


In [11]:
df.to_csv('Zoopla_rent-data2.csv')

In [12]:
data = codes.loc[231:]
data

,Postcode district,Local Areas,Borough
231,W11,"Notting Hill, Holland Park",Kensington and Chelsea
232,W12,"Shepherds Bush, White City",Hammersmith and Fulham
233,W13,"West Ealing, Northfields",Ealing
234,W14,"West Kensington, Kensington Olympia, Holland Park",Kensington and Chelsea
235,W1B,"Portland Street, Regent Street",Westminster
236,W1C,Oxford Street (west),Westminster
237,W1D,"Soho (south east), Chinatown, Soho Square",Westminster
238,W1F,Soho (north west),Westminster
239,W1G,Harley Street,Westminster
240,W1H,Marylebone,Westminster


In [ ]:
import undetected_chromedriver as webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# Set the path to the ChromeDriver executable
s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")

# Launch the ChromeDriver with the specified service
driver = webdriver.Chrome(service= s)

# Navigate to the specified URL 
driver.get('https://www.zoopla.co.uk/') 

# Click to cancel the pop-up window and maximize the window
#driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div[2]/button').click()
time.sleep(2)
driver.maximize_window()

# Find the search bar  
time.sleep(1.2)
search = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[1]/div/div/div/div/div/div/input')
search.send_keys('BR1')
time.sleep(1.3)

# Click the search button
time.sleep(3)
driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[2]/button').click()

time.sleep(3)
Trans_type = []
address = []
types = []
bedrooms = []
bathrooms = []
prices = []
desc = []
date_added = []
agent_list = []
property_url = []
website = []

address_list =driver.find_elements(By.XPATH, "//h2[contains(@class,'_1ankud51 _1ftx2fq8')]")
type_list =driver.find_elements(By.XPATH, "//h3[contains(@class,'_1ankud52 _1ftx2fq9')]")
bedroom_list =driver.find_elements(By.XPATH, "//div[contains(@id,'listing')]//li[1]//span[2]")
bathroom_list =driver.find_elements(By.XPATH, "//div[contains(@id,'listing')]//li[3]//span[2]")
price_list =driver.find_elements(By.XPATH, "//p[contains(@class,'_170k6632 _1ftx2fq6')]")
desc_list =driver.find_elements(By.XPATH, "//p[contains(@class,'_1ankud53 _1ftx2fq9')]")
date_added_list =driver.find_elements(By.XPATH, "//li[contains(@class,'_18cib8e1')]")
agent_list_list =driver.find_elements(By.XPATH, "//img[contains(@class,'_12bxhf70')]")

for ad in address_list:
    address.append(ad.text)
print(address)

                          
for ad in type_list:
    types.append(ad.text)
print(types)                                  

for ad in bedroom_list:
    bedrooms.append(ad.text)
print(bedrooms)
                              
for ad in bathroom_list:
    bathrooms.append(ad.text)
print(bathrooms)


for ad in price_list:
    prices.append(ad.text)
print(prices)


for ad in desc_list:
    desc.append(ad.text)
print(desc)


for ad in date_added_list:
    date_added.append(ad.text)
print(date_added)

for ad in agent_list_list:
    agent_list.append(ad.get_attribute('alt'))
print(agent_list)

print(len(address))
print(len(types))
print(len(bedrooms))
print(len(bathrooms))
print(len(prices))
print(len(desc))
print(len(date_added))
print(len(agent_list))

url = 'https://www.zoopla.co.uk/for-sale/property/br1/?q=BR1&search_source=home&pn=2'
if len(address_list) and len(type_list) != 0:
    driver.get(url)
else:
    print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)